In [10]:
# Necessary
import os
import sys
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, random_split
import torchvision
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from torchdiffeq import odeint_adjoint as odeint
from jupyterthemes import jtplot
from utils import *
import json
import pandas as pd
from pandas import  json_normalize 
jtplot.style(theme="chesterish")
 # CONSTANT 
device = "cuda"
EPOCHS=1
BATCH_SIZE=32
IMG_SIZE=(32,32,3)

In [35]:
result = json.loads(open("result/evaluate.json", "r").readline())
ode_result = pd.DataFrame.from_dict(result["ode"]) 
cnn_result = pd.DataFrame.from_dict(result["cnn"])


In [36]:
ode_result.mean()

null     0.996217
1e-07    0.958917
50.0     0.901047
75.0     0.790557
100.0    0.627840
dtype: float64

In [47]:
def table_(result_):
    means = []
    stds = []
    for column in result_:
#        print(str(column))
#        result_[column].astype('float64')
#        print(result_[column].describe())
#        print("\n\n")
        des = result_[column].describe()
       # print(des["mean"])
        means.append(des["mean"])
        stds.append(des["std"])
    return means,stds
ode_means, ode_std = table_(ode_result)
cnn_means, cnn_std = table_(cnn_result)

In [53]:
with open("result/mnist.md", "w") as f:
    f.write("| None | 1e-7 | 50.0 | 75.0 | 100.0 |\n")
    f.write("| :--- | :----: | :----: | :-----: | :-----: |\n")
    f.write(f"| ODENet | {round(ode_means[0],3)}$\pm${round(ode_std[0],3)}| {round(ode_means[1],3)}$\pm${round(ode_std[1],3)} | {round(ode_means[2],3)}$\pm${round(ode_std[2],3)} | {round(ode_means[3],3)}$\pm${round(ode_std[3],3)} | {round(ode_means[4],3)}$\pm${round(ode_std[4],3)} |\n")
    f.write(f"| CNN | {round(cnn_means[0],3)}$\pm${round(cnn_std[0],3)}| {round(cnn_means[1],3)}$\pm${round(cnn_std[1],3)} | {round(cnn_means[2],3)}$\pm${round(cnn_std[2],3)} | {round(cnn_means[3],3)}$\pm${round(cnn_std[3],3)} | {round(cnn_means[4],3)}$\pm${round(cnn_std[4],3)} |")


```bash
python3 run.py --parallel true --device cuda --epoch 100 --batch-size 128
```

| None | 1e-7 | 50.0 | 75.0 | 100.0 |
| :--- | :----: | :----: | :-----: | :-----: |
| ODENet | 0.996$\pm$0.001| 0.959$\pm$0.024 | 0.901$\pm$0.058 | 0.791$\pm$0.107 | 0.628$\pm$0.139 |
| CNN | 0.998$\pm$0.002| 0.612$\pm$0.203 | 0.45$\pm$0.165 | 0.351$\pm$0.115 | 0.283$\pm$0.078 |